In [0]:
import os
import pandas as pd
# from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import plotly.express as px
# import plotly
from scipy.stats import iqr 
from itertools import permutations,product
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from dateutil import relativedelta
import warnings
warnings.filterwarnings("ignore")

In [0]:
os.getcwd()
os.chdir("/dbfs/mnt/datalake/exploratory/Furnace_Analytics")

In [0]:
dcs_2018 = pd.read_csv("60TPD/dcs_2018_minutewise.csv", infer_datetime_format=True)
daily_data = pd.read_csv("60TPD/daily_data.csv", infer_datetime_format=True)
dcs_2018["Timestamp"] = dcs_2018["Timestamp"].apply(lambda x: pd.Timestamp(x))
dcs_2018["Date"] = dcs_2018["Timestamp"].apply(lambda x: str(x.date()))
dcs_2018["Time"] = dcs_2018["Timestamp"].apply(lambda x: str(x.time()))
dcs_2018 = dcs_2018.sort_values(by = "Timestamp", ascending=True)

In [0]:
dcs_2018 = dcs_2018.groupby("Date", as_index = False).agg({"APT-SA.PV" :"mean","DPT-CHMN.PV" :"mean","DPT-FUR.PV" :"mean","FT-MGF.MV" :"mean","FT-MGF.PV" :"mean"
                                                                        ,"FT-MGF.SV" :"mean","FT-MOF.MV" :"mean","FT-MOF.PV" :"mean","FT-MOF.SV" :"mean","FT-SAF.MV" :"mean","FT-SAF.PV" :"mean"
                                                                        ,"FT-SAF.SV" :"mean","TC-CH-L.PV" :"mean","TC-CH-R.PV" :"mean","TC-CHMN.PV" :"mean","TC-FP-LI.PV" :"mean"
                                                                        ,"TC-FP-LO.PV" :"mean","TC-FP-RI.PV" :"mean","TC-FP-RO.PV" :"mean","TC-MB1.PV" :"mean","TC-MB2.PV" :"mean"
                                                                        ,"TC-MB3.PV" :"mean","TC-MB4.PV" :"mean","TC-MB5.PV" :"mean","TC-MC1.PV" :"mean","TC-MC2.PV" :"mean"
                                                                        ,"TC-MC3.PV" :"mean","TC-RC-L.PV" :"mean","TC-RC-R.PV" :"mean"})

dcs_2018["air_to_fuel"] = dcs_2018["FT-SAF.PV"]/dcs_2018["FT-MGF.PV"]
dcs_2018.head()

,Date,APT-SA.PV,DPT-CHMN.PV,DPT-FUR.PV,FT-MGF.MV,FT-MGF.PV,FT-MGF.SV,FT-MOF.MV,FT-MOF.PV,FT-MOF.SV,FT-SAF.MV,FT-SAF.PV,FT-SAF.SV,TC-CH-L.PV,TC-CH-R.PV,TC-CHMN.PV,TC-FP-LI.PV,TC-FP-LO.PV,TC-FP-RI.PV,TC-FP-RO.PV,TC-MB1.PV,TC-MB2.PV,TC-MB3.PV,TC-MB4.PV,TC-MB5.PV,TC-MC1.PV,TC-MC2.PV,TC-MC3.PV,TC-RC-L.PV,TC-RC-R.PV,air_to_fuel
0,2018-01-01,0,90,-10,51.935806,392.730701,402.693866,15.000000,1736.111206,225.0,52.382674,2697.997281,2697.996933,359.703607,334.608829,438.158184,449.038595,446.825652,418.971738,417.235965,1329.872908,1323.065305,1332.217445,1150.618892,1064.016462,1523.755462,1530.921647,1567.558075,1347.842785,1354.308172,6.869841
1,2018-01-02,0,90,-10,52.697309,399.297204,409.309028,15.147557,1732.996655,225.0,46.771403,2742.812567,2742.276157,352.645944,341.430115,449.513381,436.881124,433.948158,423.257072,419.793627,1257.948120,1247.255279,1326.905589,1144.767462,1048.764156,1528.236823,1537.770264,1563.032441,1359.689013,1365.478598,6.869100
2,2018-01-03,0,90,-10,53.992466,395.655253,406.122106,15.000000,1733.499002,225.0,48.521118,2721.942648,2720.868750,356.997763,347.133091,455.667947,438.370688,446.283228,432.307831,429.498270,1258.841934,1248.146381,1282.903213,1144.162394,1042.469189,1530.945508,1541.423121,1563.661746,1357.267332,1363.592507,6.879582
3,2018-01-04,0,90,-10,55.070166,416.074097,427.109375,15.000000,1731.690553,225.0,51.417924,2874.172101,2873.639641,375.061397,363.960536,477.317690,458.953740,466.262214,460.935689,451.821314,1258.216058,1248.825439,1283.415401,1147.604483,1057.624185,1517.234435,1527.131615,1564.247281,1335.931222,1356.780156,6.907837
4,2018-01-05,0,90,-10,55.238343,414.502573,425.161748,15.000000,1731.188206,225.0,59.135951,2848.607427,2848.448322,380.461301,361.436053,480.982637,461.210020,469.526027,449.807667,443.439209,1263.926448,1254.273464,1289.349528,1150.684096,1061.014687,1520.465201,1533.127315,1565.113049,1340.192885,1362.566582,6.872352


In [0]:
daily_data = daily_data.merge(dcs_2018, how = "left", on = "Date")
daily_data = daily_data.dropna()
daily_data.head()

,Date,Melter_SCM,Boosting_KWh,DFH_SCM,GAIL_kCal,Seed_Threshold,Seed Count Result,Quantity,Total_CV,SFC,APT-SA.PV,DPT-CHMN.PV,DPT-FUR.PV,FT-MGF.MV,FT-MGF.PV,FT-MGF.SV,FT-MOF.MV,FT-MOF.PV,FT-MOF.SV,FT-SAF.MV,FT-SAF.PV,FT-SAF.SV,TC-CH-L.PV,TC-CH-R.PV,TC-CHMN.PV,TC-FP-LI.PV,TC-FP-LO.PV,TC-FP-RI.PV,TC-FP-RO.PV,TC-MB1.PV,TC-MB2.PV,TC-MB3.PV,TC-MB4.PV,TC-MB5.PV,TC-MC1.PV,TC-MC2.PV,TC-MC3.PV,TC-RC-L.PV,TC-RC-R.PV,air_to_fuel
137,2018-11-03,9782.30,8584.18,3784.44,9026.537917,30.0,13.0,49687.0,1.298431e+08,2613.220517,0.0,90.0,-10.0,53.902154,395.999077,406.390046,15.0,1735.307451,225.0,44.228502,2722.412581,2722.604745,360.042631,344.203904,458.248278,442.174297,450.480430,427.870895,427.166232,1257.431086,1248.472041,1282.484107,1146.304637,1055.549100,1526.008170,1536.149811,1564.188548,1366.191873,1367.591181,6.874795
138,2018-11-04,9877.29,8058.52,3806.40,9061.382500,30.0,17.0,49518.0,1.309235e+08,2643.957274,0.0,90.0,-10.0,55.652305,426.456851,437.533275,15.0,1735.709328,225.0,51.104163,2931.403966,2931.249016,388.553195,373.454018,487.615443,474.222003,476.980050,466.969835,463.120054,1259.043178,1247.643799,1283.665978,1149.579923,1059.326170,1515.939016,1525.420617,1563.607743,1335.495386,1360.004643,6.873858
139,2018-11-05,9885.44,8933.82,3725.40,9063.073542,30.0,15.0,51554.0,1.310391e+08,2541.783937,0.0,90.0,-10.0,55.415393,418.101182,428.904514,15.0,1734.202287,225.0,50.714826,2873.301750,2873.475347,390.283239,371.420994,496.242775,475.489005,480.735594,463.199492,450.118807,1259.737476,1248.999543,1284.968816,1148.728857,1059.455933,1514.779813,1523.142616,1564.072928,1330.840326,1356.611139,6.872264
140,2018-11-06,9940.42,12625.27,3643.39,8983.077500,30.0,14.0,56985.0,1.328822e+08,2331.879445,0.0,90.0,-10.0,54.119319,398.512357,408.876157,15.0,1720.237041,225.0,58.004223,2739.448496,2739.447917,362.975649,345.100398,461.083483,442.091403,448.719990,430.457112,419.258686,1261.353167,1252.504425,1287.677119,1142.892370,1051.210246,1530.133806,1542.418652,1566.039846,1345.543723,1368.547576,6.874187
141,2018-11-07,9711.49,15607.21,3549.97,9003.955521,30.0,27.0,58082.0,1.328278e+08,2286.901219,0.0,90.0,-10.0,55.317876,423.135019,434.139178,15.0,1723.753470,225.0,51.539447,2908.112635,2908.445370,373.393226,364.978647,490.073289,451.454748,462.135232,455.252100,448.340188,1259.185165,1248.738688,1286.465456,1141.421196,1044.502358,1525.915919,1540.105839,1563.414957,1325.316668,1368.467611,6.872777


In [0]:
# daily_data = daily_data.dropna()
# daily_data.tail()
# px.line(daily_data, x = "Date", y = "TC-MB3.PV")
px.scatter(daily_data, x = "DPT-FUR.PV", y = "SFC", color = "air_to_fuel",size = "Seed Count Result", trendline="ols", title = "SFC vs Draw vs ATF vs Seed")

In [0]:
dcs_2018.head()

,Date,APT-SA.PV,DPT-CHMN.PV,DPT-FUR.PV,FT-MGF.MV,FT-MGF.PV,FT-MGF.SV,FT-MOF.MV,FT-MOF.PV,FT-MOF.SV,FT-SAF.MV,FT-SAF.PV,FT-SAF.SV,TC-CH-L.PV,TC-CH-R.PV,TC-CHMN.PV,TC-FP-LI.PV,TC-FP-LO.PV,TC-FP-RI.PV,TC-FP-RO.PV,TC-MB1.PV,TC-MB2.PV,TC-MB3.PV,TC-MB4.PV,TC-MB5.PV,TC-MC1.PV,TC-MC2.PV,TC-MC3.PV,TC-RC-L.PV,TC-RC-R.PV,air_to_fuel
0,2018-01-01,0,90,-10,51.935806,392.730701,402.693866,15.000000,1736.111206,225.0,52.382674,2697.997281,2697.996933,359.703607,334.608829,438.158184,449.038595,446.825652,418.971738,417.235965,1329.872908,1323.065305,1332.217445,1150.618892,1064.016462,1523.755462,1530.921647,1567.558075,1347.842785,1354.308172,6.869841
1,2018-01-02,0,90,-10,52.697309,399.297204,409.309028,15.147557,1732.996655,225.0,46.771403,2742.812567,2742.276157,352.645944,341.430115,449.513381,436.881124,433.948158,423.257072,419.793627,1257.948120,1247.255279,1326.905589,1144.767462,1048.764156,1528.236823,1537.770264,1563.032441,1359.689013,1365.478598,6.869100
2,2018-01-03,0,90,-10,53.992466,395.655253,406.122106,15.000000,1733.499002,225.0,48.521118,2721.942648,2720.868750,356.997763,347.133091,455.667947,438.370688,446.283228,432.307831,429.498270,1258.841934,1248.146381,1282.903213,1144.162394,1042.469189,1530.945508,1541.423121,1563.661746,1357.267332,1363.592507,6.879582
3,2018-01-04,0,90,-10,55.070166,416.074097,427.109375,15.000000,1731.690553,225.0,51.417924,2874.172101,2873.639641,375.061397,363.960536,477.317690,458.953740,466.262214,460.935689,451.821314,1258.216058,1248.825439,1283.415401,1147.604483,1057.624185,1517.234435,1527.131615,1564.247281,1335.931222,1356.780156,6.907837
4,2018-01-05,0,90,-10,55.238343,414.502573,425.161748,15.000000,1731.188206,225.0,59.135951,2848.607427,2848.448322,380.461301,361.436053,480.982637,461.210020,469.526027,449.807667,443.439209,1263.926448,1254.273464,1289.349528,1150.684096,1061.014687,1520.465201,1533.127315,1565.113049,1340.192885,1362.566582,6.872352


In [0]:
date = "2018-11-18"
start = "07:00:00"
end = "12:00:00"
temps = ["Date","Time","TC-RC-L.PV","TC-RC-R.PV"]

tmp_265 = dcs_2018.loc[(dcs_2018.Date == date) & (dcs_2018.Time >= start) & (dcs_2018.Time <= end),temps]        
tmp_265["minute"] = tmp_265["Time"].astype(str).apply(lambda x: int(x[3:5]))
tmp_265["hour"] = tmp_265["Time"].astype(str).apply(lambda x: int(x[:2]))
tmp_265["regen_left_diffrential"] = tmp_265["TC-RC-L.PV"].diff(1)
tmp_265["regen_right_diffrential"] = tmp_265["TC-RC-R.PV"].diff(1)
tmp_265.head()

,Date,Time,TC-RC-L.PV,TC-RC-R.PV,minute,hour,regen_left_diffrential,regen_right_diffrential
308560,2018-11-18,07:00:56,1366.973897,1347.689178,0,7,NaN,NaN
308561,2018-11-18,07:01:56,1360.231761,1353.531281,1,7,-6.742137,5.842102
308562,2018-11-18,07:02:56,1355.139740,1358.764964,2,7,-5.092021,5.233683
308563,2018-11-18,07:03:56,1351.206106,1362.723602,3,7,-3.933634,3.958639
308564,2018-11-18,07:04:56,1348.222555,1365.857157,4,7,-2.983551,3.133555


In [0]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# TE104TC26,TE105TC27
# regen_left_diffrential,regen_right_diffrential
# Add traces
fig.add_trace(
    go.Scatter(x=tmp_265["Time"], y=tmp_265["TC-RC-L.PV"], name="Regen Left ", mode = "lines"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=tmp_265["Time"], y=tmp_265["TC-RC-R.PV"], name="Regen Right "),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Regen Left vs Right Diff | 60 TPD | Nov 18th 2018"
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Regen Left </b>", secondary_y=False, range = [1315,1410])
fig.update_yaxes(title_text="<b>Regen Right </b>", secondary_y=True, range = [1315,1410])

fig.show()